In [115]:
import requests
import lxml.html as lh
import json
import time

base_url = "http://xc.rmhpa.org/tracks/world/alltimes/brand:all,cat:1,class:all,xctype:all,club:all,pilot:0_0,takeoff:all&sortOrder=FLIGHT_POINTS"

def process_url(page=None):
    time.sleep(1)
    url = base_url
    if page is not None:
        url = base_url + "&page_num={}".format(page)
        
    res = requests.get(url)

    doc = lh.fromstring(res.content)
    names = ['index', 'date', 'name location', 'duration', 'distance', 'xckm', 'xcscore', 'cat', 'wing', 'links']

    data = []
    for tbl in doc.xpath('//table'):
        tr_elements = tbl.xpath('.//tr')
        for td_element in tr_elements:
            cols = td_element.xpath('.//td')
            if len(cols)==10:
                tmp = {}
                for name, col in zip(names, cols):
                    if name=='name location':
                        n, l = col.text_content().strip().split("\n")
                        tmp['name'] = n
                        tmp['location'] = l
                    elif name == 'wing':
                        tmp['wing'] = col.xpath('.//img')[0].get('title')
                    elif name == 'links':
                        tmp['links'] = "http://xc.rmhpa.org" + col.xpath('.//a')[0].get('href')
                    elif name == 'cat':
                        pass
                    else:
                        tmp[name] = col.text_content().strip()
                data.append(tmp)
    
    if page is None:
        page = 0
    
    with open("raw/data_{}.json".format(page), 'w') as outfile:
        outfile.write(json.dumps(data, indent=4))

In [117]:
for page in range(1, 27):
    process_url(page)